In [16]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib as plt

#Import cranial database 
df = pd.read_csv('../../datasources/fdb/cranial.csv')
#This shuffles the rows.
df = df.sample(frac=1)

#Drop categorical data. We'll focus only on measurement data.
del df['DB']
del df['Item']
del df['ID']
del df['ContNum']
del df['FDN']
del df['Pop']
del df['PopSex']
del df['Ethnicity'] #Probably really important.
del df['BirthYear']
del df['Age'] #Probably really important.
del df['Comments']

#This col is basically empty and disturbs standardization.
del df["MOW"]

rows, cols = df.shape 
print("Starting with", rows, "individuals and", cols, "features.")

Starting with 5342 individuals and 110 features.


In [17]:
#Consider our target col, Sex.
print("Unique entries.")
df['Sex'].value_counts(dropna=False)

Unique entries.


M      3187
F      2095
N        58
NaN       2
Name: Sex, dtype: int64

In [18]:
#Clean N and Nan entries
df = df[~(df.Sex.str.contains("N") == True)]
df = df.dropna(subset = ['Sex'])

#Move from object to float.
df['Sex'] = df['Sex'].map({'M':1.0, 'F':0.})
df.describe()

,Sex,GOL,NOL,BNL,BBH,XCB,XFB,WFB,ZYB,AUB,...,MLN,MAN,BABR,BANA,BAPR,UFHT,UFBR,ORBR,BIOB,INTB
count,5282.000000,5200.000000,4396.000000,5177.000000,5182.000000,5196.000000,4395.000000,2622.000000,5096.000000,5106.000000,...,519.000000,518.000000,636.000000,607.000000,574.000000,2115.000000,1605.000000,453.000000,604.000000,413.000000
mean,0.603370,180.397500,177.689945,100.143906,133.846584,137.568129,114.752673,94.898932,129.449372,120.855660,...,77.046243,124.990347,136.886792,101.494234,97.031359,70.201891,103.535826,41.008830,96.253311,20.552058
std,0.489244,8.838111,8.238570,5.764435,7.336864,6.811824,6.477442,4.952665,7.313785,6.763771,...,7.211489,7.455211,6.889893,5.523686,6.481473,4.934519,4.827782,2.493338,4.400408,3.388348
min,0.000000,131.000000,150.000000,79.000000,107.000000,116.000000,93.000000,79.000000,90.000000,98.000000,...,54.000000,90.000000,84.000000,85.000000,73.000000,47.000000,87.000000,34.000000,81.000000,0.000000
25%,0.000000,174.000000,172.000000,96.000000,129.000000,133.000000,110.000000,92.000000,124.000000,116.000000,...,73.000000,120.250000,133.000000,98.000000,93.000000,67.000000,100.000000,39.000000,93.000000,19.000000
50%,1.000000,180.000000,178.000000,100.000000,134.000000,137.000000,115.000000,95.000000,129.000000,121.000000,...,77.000000,125.000000,137.000000,101.000000,97.000000,70.000000,104.000000,41.000000,96.000000,21.000000
75%,1.000000,187.000000,183.000000,104.000000,139.000000,142.000000,119.000000,98.000000,134.000000,125.000000,...,81.000000,130.000000,142.000000,105.000000,101.000000,74.000000,107.000000,42.000000,99.000000,22.000000
max,1.000000,211.000000,207.000000,120.000000,157.000000,167.000000,145.000000,119.000000,158.000000,154.000000,...,120.000000,147.000000,156.000000,130.000000,116.000000,91.000000,122.000000,51.000000,112.000000,29.000000


In [19]:
#Split into data and labels.
X = df.drop(['Sex'], axis=1)
y = df[['Sex']]

In [20]:
#Split train test
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01)

In [21]:
X_train.isnull().sum().sum()

200836

In [22]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5229, 109), (53, 109), (5229, 1), (53, 1))

In [23]:
#Standardize df. Can't use scikit's scale because it doesn't like Nan.
X_train_mean = X_train.mean()
X_train_std = X_train.std()
X_train = (X_train - X_train_mean) / (X_train_std)
#Intentionally using train stats for test set.
X_test = ( X_test - X_train_mean) / (X_train_std)
X_train.describe()

,GOL,NOL,BNL,BBH,XCB,XFB,WFB,ZYB,AUB,ASB,...,MLN,MAN,BABR,BANA,BAPR,UFHT,UFBR,ORBR,BIOB,INTB
count,5.149000e+03,4.354000e+03,5.125000e+03,5.130000e+03,5.145000e+03,4.354000e+03,2.595000e+03,5.044000e+03,5.056000e+03,4.356000e+03,...,5.110000e+02,5.110000e+02,6.260000e+02,5.970000e+02,5.650000e+02,2.093000e+03,1.593000e+03,4.440000e+02,5.930000e+02,4.050000e+02
mean,-7.484788e-16,1.464913e-15,-9.256985e-16,-1.634993e-15,9.568418e-16,1.205538e-15,3.566404e-16,1.287674e-15,-2.546750e-16,7.072065e-16,...,-3.802134e-18,-1.051562e-16,1.215199e-15,-8.594465e-16,3.880868e-17,-1.183371e-15,6.183238e-16,-1.268880e-15,-1.021855e-15,1.436437e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-5.592392e+00,-3.366252e+00,-3.673726e+00,-3.658135e+00,-3.165778e+00,-3.358941e+00,-3.213419e+00,-5.395259e+00,-3.378691e+00,-3.360803e+00,...,-3.172503e+00,-4.675418e+00,-7.680145e+00,-2.985927e+00,-3.716070e+00,-4.700620e+00,-3.428648e+00,-2.802876e+00,-3.457685e+00,-6.331660e+00
25%,-7.252169e-01,-6.924329e-01,-7.195752e-01,-6.591148e-01,-6.688461e-01,-7.325772e-01,-5.828118e-01,-7.456611e-01,-7.172185e-01,-7.291317e-01,...,-5.535223e-01,-6.650019e-01,-5.577370e-01,-6.291916e-01,-6.273174e-01,-6.489788e-01,-7.328785e-01,-7.982417e-01,-7.292656e-01,-4.910794e-01
50%,-4.607622e-02,3.679056e-02,-2.448088e-02,2.248069e-02,-8.133265e-02,3.988263e-02,2.425133e-02,-6.189667e-02,2.207951e-02,-7.121400e-02,...,-1.399991e-01,3.400875e-03,2.368409e-02,-8.532956e-02,-9.566932e-03,-4.123266e-02,9.658895e-02,3.611953e-03,-4.716071e-02,1.237186e-01
75%,7.462545e-01,6.444768e-01,6.706134e-01,7.040762e-01,6.530592e-01,6.578505e-01,6.313145e-01,6.218677e-01,6.135179e-01,5.867037e-01,...,5.492063e-01,6.718036e-01,7.504605e-01,6.398198e-01,6.081835e-01,7.690956e-01,7.186895e-01,4.045388e-01,6.349442e-01,4.311176e-01
max,3.462817e+00,3.561371e+00,3.450991e+00,3.157820e+00,4.325018e+00,4.674642e+00,4.880756e+00,3.903937e+00,4.901446e+00,3.547333e+00,...,5.925008e+00,2.944373e+00,2.785434e+00,5.172004e+00,2.924748e+00,4.212990e+00,3.829192e+00,4.012880e+00,3.590732e+00,2.582910e+00


In [24]:
#The imputer. I haven't looked closely at this algo. Try other variations.
from fancyimpute import KNN, SoftImpute
# X_train_filled = pd.DataFrame(KNN(k=3).complete(X_train))
# X_test_filled = pd.DataFrame(KNN(k=3).complete(X_test))
X_train_filled = SoftImpute().complete(X_train)
X_test_filled = SoftImpute().complete(X_test)

[SoftImpute] Max Singular Value of X_init = 279.492252
[SoftImpute] Iter 1: observed MAE=0.061960 rank=106
[SoftImpute] Iter 2: observed MAE=0.062372 rank=103
[SoftImpute] Iter 3: observed MAE=0.062646 rank=102
[SoftImpute] Iter 4: observed MAE=0.062866 rank=101
[SoftImpute] Iter 5: observed MAE=0.063029 rank=101
[SoftImpute] Iter 6: observed MAE=0.063177 rank=101
[SoftImpute] Iter 7: observed MAE=0.063287 rank=100
[SoftImpute] Iter 8: observed MAE=0.063375 rank=99
[SoftImpute] Iter 9: observed MAE=0.063450 rank=99
[SoftImpute] Iter 10: observed MAE=0.063514 rank=99
[SoftImpute] Iter 11: observed MAE=0.063568 rank=99
[SoftImpute] Iter 12: observed MAE=0.063613 rank=99
[SoftImpute] Iter 13: observed MAE=0.063643 rank=98
[SoftImpute] Iter 14: observed MAE=0.063660 rank=97
[SoftImpute] Iter 15: observed MAE=0.063677 rank=97
[SoftImpute] Iter 16: observed MAE=0.063692 rank=97
[SoftImpute] Iter 17: observed MAE=0.063706 rank=97
[SoftImpute] Iter 18: observed MAE=0.063717 rank=97
[SoftImpute

In [25]:
#Num zero entries
(X_train_filled == 0).astype(int).sum().sum(),(X_test_filled == 0).astype(int).sum().sum()

(1962, 0)

In [26]:
#Impute zeros with mean. Could do this better.
#Think this introduces an improbable bug when the train set
#Has representation in a col that the test set doesn't or 
#Vice versa. Ignoring it for now. 
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values=0, strategy='mean', axis=0)
X_train_filled = pd.DataFrame(imp.fit_transform(X_train_filled))
X_train_filled = pd.DataFrame(imp.transform(X_train_filled))
X_test_filled = pd.DataFrame(imp.fit_transform(X_test_filled))

In [27]:
#No zero values
(X_train_filled == 0).astype(int).sum().sum(), (X_test_filled == 0).astype(int).sum().sum()

(0, 0)

In [28]:
#Notice that the training (and testing) set is no longer standardized
X_train_filled.describe()

,0,1,2,3,4,5,6,7,8,9,...,99,100,101,102,103,104,105,106,107,108
count,5229.000000,5229.000000,5229.000000,5229.000000,5229.000000,5229.000000,5229.000000,5229.000000,5229.000000,5229.000000,...,5229.000000,5229.000000,5229.000000,5229.000000,5229.000000,5229.000000,5229.000000,5229.000000,5229.000000,5229.000000
mean,0.002345,0.025971,0.002379,0.002190,0.000950,0.010316,-0.035145,-0.002372,-0.002103,0.010693,...,-0.004966,-0.004903,-0.010775,-0.002039,-0.003513,-0.032981,-0.002816,0.005193,-0.002352,0.000726
std,0.994510,0.976106,0.994934,0.993020,0.993087,0.938528,0.800580,0.988857,0.988110,0.934795,...,0.403905,0.346510,0.601012,0.624881,0.652541,0.897025,0.811173,0.453427,0.638914,0.383673
min,-5.592392,-3.366252,-3.673726,-3.658135,-3.165778,-3.358941,-3.213419,-5.395259,-3.378691,-3.360803,...,-3.172503,-4.675418,-7.680145,-2.985927,-3.716070,-4.700620,-3.428648,-2.802876,-3.457685,-6.331660
25%,-0.725217,-0.612526,-0.719575,-0.659115,-0.668846,-0.578085,-0.540047,-0.745661,-0.717218,-0.564652,...,-0.196844,-0.108455,-0.391959,-0.414420,-0.427490,-0.611771,-0.525512,-0.263802,-0.415893,-0.183680
50%,-0.046076,0.036791,-0.024481,0.022481,-0.081333,0.039883,-0.050121,-0.061897,0.022080,0.010693,...,-0.004912,-0.003000,-0.010579,-0.014333,-0.009567,-0.041233,-0.002816,0.003612,-0.015786,0.000676
75%,0.746255,0.644477,0.670613,0.704076,0.653059,0.611372,0.428960,0.621868,0.613518,0.586704,...,0.179587,0.101647,0.376633,0.395908,0.406248,0.566514,0.511323,0.259040,0.400575,0.182503
max,3.462817,3.561371,5.605809,3.157820,4.325018,4.674642,4.880756,3.903937,4.901446,3.547333,...,5.925008,2.944373,2.785434,5.172004,2.924748,4.212990,3.829192,4.012880,3.590732,2.582910


In [29]:
#Should we standardize again?
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_train_filled)
X_train_filled = pd.DataFrame(scaler.transform(X_train_filled))
#Uses test stats
X_test_filled = pd.DataFrame(scaler.transform(X_test_filled))

In [30]:
#Save to disk
X_train_filled.to_csv("data/X_train_preproc.csv", index=False)
#Ravel strips col name and reindexes
pd.DataFrame(np.ravel(y_train)).to_csv("data/y_train_preproc.csv", index=False)

X_test_filled.to_csv("data/X_test_preproc.csv", index=False)
pd.DataFrame(np.ravel(y_test)).to_csv("data/y_test_preproc.csv", index=False)